In [1309]:
from PIL import Image 
import math

In [1380]:
center = (0, 0, 40)
radius = 8
color = (0, 255, 0) 
specular = 700
intensity = 1

d = 100
Vw = 100
Vh = 100

In [1381]:
def CanvasToViewport(x, y): 
    return (x*Vw/Cw, y*Vh/Ch, d)

In [1382]:
def length(V):
    return math.sqrt(V[0]*V[0] + V[1]*V[1] + V[2]*V[2])

In [1383]:
def dot(V1, V2):
    return V1[0]*V2[0] + V1[1]*V2[1] + V1[2]*V2[2]

In [1384]:
def ComputeLighting(P, N, V, s):
    i = 0.0
    L = (120, -100, 30)
    
    n_dot_l = dot(N, L)
    if (n_dot_l > 0):
        i += intensity * n_dot_l / (length(N)*length(L)) 
        
    if (s != -1): 
        R = (2*N[0]*n_dot_l - L[0], 2*N[1]*n_dot_l - L[1], 2*N[2]*n_dot_l - L[2])
        r_dot_v = dot(R, V)
        
        if (r_dot_v > 0):
            i += intensity * pow(r_dot_v/(length(R)*length(V)), s)    
    return i

In [1385]:
def IntersectRaySphere(O, D):
    C = center
    r = radius
    
    OC = (O[0] - C[0], O[1] - C[1], O[2] - C[2])

    k1 = dot(D, D)
    k2 = 2 * dot(OC, D)
    k3 = dot(OC, OC) - r*r

    discriminant = k2*k2 - 4*k1*k3
    if (discriminant < 0):
        return (math.inf, math.inf) 

    t1 = (-k2 + math.sqrt(discriminant)) / (2*k1)
    t2 = (-k2 - math.sqrt(discriminant)) / (2*k1)
    return (t1, t2)

In [1386]:
def TraceRay(O, D, t_min, t_max): 
    closest_t = 1000000
    
    t1 = IntersectRaySphere(O, D)[0]
    if(t1 == math.inf):
        return (50, 50, 50)
    t2 = IntersectRaySphere(O, D)[1]
    
    if (t1 in [t_min, t_max] and t1 < closest_t):
        closest_t = t1
    if (t2 in [t_min, t_max] and t2 < closest_t):
        closest_t = t2
        
    P = (O[0] + closest_t*D[0], O[1] + closest_t*D[1], O[2] + closest_t*D[2])   
    X = (P[0] - center[0], P[1] - center[1], P[2] - center[2] )
    lenN = length(X)
    N = (X[0] / lenN, X[1] / lenN, X[2] / lenN) 
    T = (-D[0], -D[1], -D[2])
    
    return (255*ComputeLighting(P, N, T, 700),0 , 0)

In [1387]:
image = Image.open(r'Scene1.png')  
  
Cw, Ch = image.size 
O = (0, 0, 0)  
    
for x in range(-400, 400):
    x1 = x + 400
    for y in range(-400, 400):
        y1 = y + 400
        color = TraceRay(O, CanvasToViewport(x, y), 1, math.inf)
        image.putpixel((x1, y1), tuple(int(c) for c in color) )  
image.show() 